In [ ]:
import sys
sys.path.append('../src')

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

___

In [ ]:
# random
n_assets = 4
n_obs = 1000
sigma = 4 / np.sqrt(252)
returns = pd.DataFrame(sigma*np.random.randn(n_obs, n_assets), columns=['a_1','a_2', 'a_3', 'a_4'])

In [ ]:
mu = np.mean(returns, axis=0)
cov = returns.cov()

In [ ]:
sns.heatmap(cov, annot=True, fmt=".2f")

### Montecarlo simulation
___

In [ ]:
n_sim = 500

In [ ]:
w = np.random.rand(n_sim, n_assets)
w = w / w.sum(axis=1, keepdims=True)

In [ ]:
r = w.dot(mu.T)
std = np.apply_along_axis(lambda x: np.sqrt(x.dot(cov).dot(x.T)), 1, w)

In [ ]:
w_one = np.diagflat(np.ones(4))
r_ones = w_one.dot(mu.T)
std_ones = np.apply_along_axis(lambda x: np.sqrt(x.dot(cov).dot(x.T)), 1, w_one)

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
ax.plot(std, r, 'o', markersize=5)
#ax.plot(returns.std(), returns.mean(), 'o', markersize=5)
#ax.plot(std_ones, r_ones, 'o', markersize=5)
_ = plt.xlabel('std')
_ = plt.ylabel('r')

### Efficient Frontier
___

In [ ]:
from port_opt.efficient_frontier import efficient_frontier_convex

In [ ]:
ret_data, risk_data, gamma_vals, portfolio_weights = efficient_frontier_convex(returns,  n_samples=1000, gamma_low=-5, gamma_high=5)

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
ax.plot(std, r, 'o', markersize=5)
ax.plot(risk_data, ret_data, 'r', markersize=5)
_ = plt.xlabel('std')
_ = plt.ylabel('r')

### CLA
___

In [ ]:
from port_opt.efficient_frontier import efficient_frontier_cla
from port_opt.port_optimizer import PortfolioOptimizer

In [ ]:
means_cla, sigma_cla, weights_cla= efficient_frontier_cla(returns)

In [ ]:
def compute_ret_sigma(w, mu, sigma):
    r = w.dot(mu)
    std = np.sqrt(w.dot(sigma).dot(w))
    return r, std

In [ ]:
po = PortfolioOptimizer(returns)
max_sharpe_cla_weights = po.cla_max_sharpe_portfolio()
max_sharpe_cla_weights = max_sharpe_cla_weights.reindex(mu.index).fillna(0)
r_max_sharpe, std_max_sharpe = compute_ret_sigma(max_sharpe_cla_weights, mu, cov)
min_vol_cla_weights = po.cla_min_volatility_portfolio()
r_min_vol, std_min_vol = compute_ret_sigma(min_vol_cla_weights, mu, cov)

In [ ]:
fig, ax = plt.subplots(figsize=(10,7))
ax.plot(std[:30], r[:30], 'or', markersize=5)
ax.plot(std[30:], r[30:], 'o', markersize=5)

ax.plot(sigma_cla, means_cla, 'r', markersize=5)

ax.plot(std_max_sharpe, r_max_sharpe, '*', markersize=20)
ax.plot(std_min_vol, r_min_vol, '.', markersize=20)

ax.plot([0, std_max_sharpe], [0, r_max_sharpe], 'g')

ax.set_xlim(-0.008, 0.008)
ax.set_xlim(0, 0.25)

_ = plt.xlabel('std')
_ = plt.ylabel('r')